In [6]:
# Manipulación de datos
import numpy as np
from sklearn.utils import shuffle
import zipfile
import Funciones_Kaggle_ReconocimientoFacial as fc

# Carga y preprocesamiento de imágenes
import os
import cv2

# Construcción y entrenamiento del modelo
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
# from tf.keras.wrappers.scikit_learn import KerasClassifier


Vamos a comenzar descomprimiendo el archivo y luego exploraremos los datos para entender mejor el contenido y la estructura. Luego, procederemos con el preprocesamiento y el entrenamiento del modelo.

Descomprimamos el archivo primero.

In [7]:
zip_file_path = 'data.zip'
extracted_folder_path = 'data/facial_expressions'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

extracted_contents = os.listdir(extracted_folder_path)
extracted_contents

['data']

El archivo ZIP contiene una carpeta llamada data. Vamos a explorar más a fondo para ver cómo están organizadas las imágenes dentro de esta carpeta.

In [8]:
data_folder_path = os.path.join(extracted_folder_path, 'data')
data_contents = os.listdir(data_folder_path)
data_contents

['images']

Dentro de es encontramos una subcarpeta llamada images. Vamos a explorar esta subcarpeta para ver cómo están organizadas las imágenes y obtener una idea del número de imágenes disponibles.

In [9]:
images_folder_path = os.path.join(data_folder_path, 'images')
images_contents = os.listdir(images_folder_path)
num_images = len(images_contents)
images_contents[:10], num_images

(['test', 'train'], 2)

La carpeta images contiene dos subcarpetas: train y test. Exploraremos ambas carpetas, comencemos con Train.

In [10]:
train_folder_path = os.path.join(images_folder_path, 'train')
train_contents = os.listdir(train_folder_path)
num_train_images = len(train_contents)
train_contents[:10], num_train_images

(['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'], 7)

Dado que nuestro enfoque está en clasificar entre "feliz" y "triste", utilizaremos únicamente las carpetas happy y sad.

In [11]:
happy_train_folder = os.path.join(train_folder_path, 'happy')
sad_train_folder = os.path.join(train_folder_path, 'sad')

happy_train_images = os.listdir(happy_train_folder)
sad_train_images = os.listdir(sad_train_folder)

num_happy_train_images = len(happy_train_images)
num_sad_train_images = len(sad_train_images)

num_happy_train_images, num_sad_train_images

(7164, 4938)

La carpeta de entrenamiento contiene 7164 imágenes de personas felices y 4938 imágenes de personas tristes. 

Ahora exploraremos Test.

In [12]:
test_folder_path = os.path.join(images_folder_path, 'test')
test_contents = os.listdir(test_folder_path)
test_contents

['10004.jpg',
 '10019.jpg',
 '10023.jpg',
 '10029.jpg',
 '1003.jpg',
 '10031.jpg',
 '10033.jpg',
 '10043.jpg',
 '10044.jpg',
 '10048.jpg',
 '10052.jpg',
 '10053.jpg',
 '10056.jpg',
 '10065.jpg',
 '10068.jpg',
 '10073.jpg',
 '10074.jpg',
 '10079.jpg',
 '1008.jpg',
 '10095.jpg',
 '10096.jpg',
 '10097.jpg',
 '10099.jpg',
 '101.jpg',
 '10106.jpg',
 '10114.jpg',
 '10116.jpg',
 '10117.jpg',
 '10118.jpg',
 '10121.jpg',
 '10126.jpg',
 '10134.jpg',
 '10138.jpg',
 '10141.jpg',
 '10148.jpg',
 '10150.jpg',
 '10162.jpg',
 '10163.jpg',
 '10171.jpg',
 '10172.jpg',
 '10176.jpg',
 '10185.jpg',
 '10189.jpg',
 '1020.jpg',
 '10215.jpg',
 '10218.jpg',
 '1022.jpg',
 '10237.jpg',
 '1024.jpg',
 '10246.jpg',
 '10247.jpg',
 '10248.jpg',
 '10252.jpg',
 '10257.jpg',
 '10259.jpg',
 '1026.jpg',
 '10263.jpg',
 '10266.jpg',
 '10267.jpg',
 '10268.jpg',
 '1027.jpg',
 '10273.jpg',
 '10276.jpg',
 '10278.jpg',
 '10286.jpg',
 '10292.jpg',
 '10306.jpg',
 '10312.jpg',
 '10315.jpg',
 '10317.jpg',
 '1033.jpg',
 '10336.jpg',
 '

contiene una gran cantidad de imágenes, pero no están organizadas en subcarpetas de emociones. Dado que la clasificación entre "feliz" y "triste" debe realizarse, será necesario que evaluemos cómo están etiquetadas estas imágenes o si requieren una clasificación manual para el test.

Para proceder con la construcción de nuestro modelo, primero prepararemos el conjunto de entrenamiento. Realizaremos los siguientes pasos:

1. Cargar las imágenes de las carpetas happy y sad.
2. Preprocesar las imágenes (normalización, redimensionamiento si es necesario).
3. Crear las etiquetas correspondientes para cada categoría (0 para "sad" y 1 para "happy").
4. Entrenar un modelo de red neuronal convolucional (CNN) utilizando estos datos.

Vamos a comenzar con la carga y preprocesamiento de las imágenes de entrenamiento, para ello usaré el módulo de Funciones (fc).

In [13]:
happy_train_images, happy_train_labels = fc.load_images_from_folder(happy_train_folder, label=1)

In [14]:
sad_train_images, sad_train_labels = fc.load_images_from_folder(sad_train_folder, label=0)

In [15]:
X_train = np.concatenate((happy_train_images, sad_train_images), axis=0)
y_train = np.concatenate((happy_train_labels, sad_train_labels), axis=0)

In [16]:
X_train, y_train = shuffle(X_train, y_train, random_state=42)

In [17]:
X_train.shape, y_train.shape

((12102, 48, 48, 1), (12102,))

In [18]:
# Reshape the images for model compatibility
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

In [19]:
# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)

--------------------------------------------------------------------------------------------------------------------------

## Construcción y Entrenamiento del Modelo

---------------------------------------------------------------------------------------------------------------------------

In [20]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')
])

C:\Users\buque\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
